In [125]:
import torch
import torch.nn as nn
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [126]:
class housePriceModel(nn.Module):
    def __init__(self, n_features):
        super(housePriceModel, self).__init__()

        self.network = nn.Sequential(
            nn.Linear(n_features, 64),
            nn.ReLU(),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Linear(32, 1)
        )

    def forward(self, x):
        return self.network(x)

In [127]:
# Load and create data matrix and target vector
df = pd.read_pickle("cleaned_df.pkl")

In [128]:
df

,price,condominium_fees,rooms,m2,bathrooms,elevator,is_luxury,floor,energy_class,year_of_construction,...,Cancello elettrico,Esposizione doppia,Terrazza,Cantina,VideoCitofono,Impianto di allarme,Giardino privato,Caminetto,Giardino comune,Arredato
30018,413000.0,168.0,2.0,72.0,2.0,0.0,0.0,1.0,1.0,1964.0,...,1,0,0,0,1,0,0,0,0,0
30019,1062000.0,382.0,4.0,164.0,4.0,0.0,0.0,3.0,1.0,1962.0,...,1,0,0,0,1,0,0,0,0,0
30020,696000.0,254.0,3.0,109.0,3.0,0.0,0.0,2.0,1.0,1964.0,...,1,0,0,0,1,0,0,0,0,0
30021,502000.0,200.0,3.0,86.0,3.0,0.0,0.0,2.0,5.0,1978.0,...,0,0,0,0,1,1,0,0,0,0
30022,610000.0,50.0,3.0,126.0,3.0,0.0,0.0,2.0,1.0,1970.0,...,1,1,0,0,1,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
147562,189000.0,150.0,2.0,60.0,2.0,1.0,0.0,2.0,6.0,1960.0,...,1,0,0,1,0,0,0,0,1,0
147568,680000.0,500.0,5.0,160.0,5.0,1.0,0.0,3.0,5.0,1970.0,...,1,1,0,1,0,0,0,0,1,0
147569,417000.0,60.0,3.0,70.0,3.0,0.0,0.0,0.0,4.0,1950.0,...,1,1,0,0,0,0,0,0,0,0
147571,1050000.0,625.0,5.0,163.0,5.0,1.0,1.0,6.0,5.0,1970.0,...,1,0,0,1,1,1,0,0,0,1


In [129]:
X = df.drop(columns=["price"], axis=1).to_numpy(dtype="float32")

y = df["price"].to_numpy(dtype="float32")

In [130]:
# Train, validation and test splits
X_train, X_tmp, y_train, y_tmp = train_test_split(X, y, test_size=0.30, shuffle=True)
X_val, X_test, y_val, y_test = train_test_split(X_tmp, y_tmp, test_size=0.10, shuffle=True)

# Check shapes
print(f"X_train, y_train (70%): {X_train.shape}, {y_train.shape}")
print(f"X_val, y_val (20%): {X_val.shape}, {y_val.shape}")
print(f"X_test, y_test (10%): {X_test.shape}, {y_test.shape}")

X_train, y_train (70%): (4393, 24), (4393,)
X_val, y_val (20%): (1694, 24), (1694,)
X_test, y_test (10%): (189, 24), (189,)


In [131]:
X_train = torch.tensor(X_train, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.float32).view(-1, 1)

X_val   = torch.tensor(X_val, dtype=torch.float32)
y_val   = torch.tensor(y_val, dtype=torch.float32).view(-1, 1)

X_test  = torch.tensor(X_test, dtype=torch.float32)
y_test  = torch.tensor(y_test, dtype=torch.float32).view(-1, 1)


In [132]:
from torch.utils.data import TensorDataset

train_dataset = TensorDataset(X_train, y_train)
val_dataset = TensorDataset(X_val, y_val)
test_dataset = TensorDataset(X_test, y_test)

In [133]:
from torch.utils.data import DataLoader

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)  # shuffle only train
val_loader   = DataLoader(val_dataset, batch_size=32, shuffle=False)
test_loader  = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [134]:
# Initialize the model
n_features = X.shape[1]

model = housePriceModel(n_features)
model

housePriceModel(
  (network): Sequential(
    (0): Linear(in_features=24, out_features=64, bias=True)
    (1): ReLU()
    (2): Linear(in_features=64, out_features=32, bias=True)
    (3): ReLU()
    (4): Linear(in_features=32, out_features=1, bias=True)
  )
)

In [135]:
import torch.optim as optim

loss_fn = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [136]:
# Training loop
n_epochs = 120
for epoch in range(n_epochs):
    model.train()
    train_loss = 0.0
    for X_batch, y_batch in train_loader:
        optimizer.zero_grad()
        y_pred = model(X_batch)
        loss = loss_fn(y_pred, y_batch)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
    
    # Validation
    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for X_batch, y_batch in val_loader:
            y_pred = model(X_batch)
            loss = loss_fn(y_pred, y_batch)
            val_loss += loss.item()
    
    # Logging
    print(f"Epoch {epoch+1}/{n_epochs} | Train Loss: {train_loss/len(train_loader):.4f} | Val Loss: {val_loss/len(val_loader):.4f}")


Epoch 1/120 | Train Loss: 1067013358755.2463 | Val Loss: 4036198214520.7549
Epoch 2/120 | Train Loss: 979784678711.6522 | Val Loss: 3858091726886.6416
Epoch 3/120 | Train Loss: 726922070639.3043 | Val Loss: 3542422722907.7734
Epoch 4/120 | Train Loss: 500988214524.2899 | Val Loss: 3417962146526.1885
Epoch 5/120 | Train Loss: 461274139366.0290 | Val Loss: 3407074705717.1318
Epoch 6/120 | Train Loss: 454981758263.6522 | Val Loss: 3401707794045.5850
Epoch 7/120 | Train Loss: 453638809689.0435 | Val Loss: 3396880351386.5659
Epoch 8/120 | Train Loss: 445413746005.3333 | Val Loss: 3391708738540.6792
Epoch 9/120 | Train Loss: 441415216320.9276 | Val Loss: 3386657596628.5283
Epoch 10/120 | Train Loss: 435963390604.9855 | Val Loss: 3381442849038.4907
Epoch 11/120 | Train Loss: 430187734090.2029 | Val Loss: 3377188035680.6040
Epoch 12/120 | Train Loss: 430223677410.3188 | Val Loss: 3372525529049.3584
Epoch 13/120 | Train Loss: 420436689192.8116 | Val Loss: 3367986201194.2642
Epoch 14/120 | Train

In [ ]:
model.eval()
test_loss = 0.0
with torch.no_grad():
    for X_batch, y_batch in test_loader:
        y_pred = model(X_batch)
        print(f"y_pred: {y_pred[5]} -> y_batch: {y_batch[5]}")
        loss = loss_fn(y_pred, y_batch)
        test_loss += loss.item()

# print(f"Test Loss: {test_loss/len(test_loader):.4f}")

y_pred: tensor([1471067.1250]) -> y_batch: tensor([2050000.])
y_pred: tensor([639843.5000]) -> y_batch: tensor([495000.])
y_pred: tensor([282590.5938]) -> y_batch: tensor([150000.])
y_pred: tensor([303396.5625]) -> y_batch: tensor([270000.])
y_pred: tensor([509728.8750]) -> y_batch: tensor([330000.])
y_pred: tensor([301777.7188]) -> y_batch: tensor([300000.])
